## Tiweeter_Sentimental_Analisys

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/



cp: cannot stat 'kaggle.json': No such file or directory


In [2]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 174MB/s]
100% 80.9M/80.9M [00:00<00:00, 145MB/s]


In [3]:

import zipfile
zip_ref = zipfile.ZipFile('/content/sentiment140.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='latin-1')
df.head(2)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                                                                                                               Non-Null Count    Dtype 
---  ------                                                                                                               --------------    ----- 
 0   0                                                                                                                    1599999 non-null  int64 
 1   1467810369                                                                                                           1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009                                                                                         1599999 non-null  object
 3   NO_QUERY                                                                                                             1599999 non-null  object
 4   _

In [7]:
df.shape

(1599999, 6)

In [8]:
# naming the col and reading the dataset again
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
df.head(2)

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [9]:
df.head(1)

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [10]:
df.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [11]:
# 0=negetive
# 2=neutral
# 4=positive
df['target'].value_counts()

,count
target,
4,800000
0,799999


In [12]:
# making 4 to 1
df['target'] = df['target'].replace(4, 1)

In [13]:
df['target'].value_counts()

,count
target,
1,800000
0,799999


In [14]:
# stemming
import nltk
nltk.download('stopwords')

ps = PorterStemmer()
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content= stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df['stemmed_text'] = df['text'].apply(stemming)

In [ ]:
df.head(2)

In [ ]:
x=df['stemmed_text'].values
y=df['target'].values

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
print(x_train.shape,x_test.shape)

In [ ]:
vector = TfidfVectorizer()
vector.fit(x_train)
x_train = vector.transform(x_train)
x_test = vector.transform(x_test)

In [ ]:
moedl = LogisticRegression()
moedl.fit(x_train,y_train)

In [ ]:
x_train_pred = moedl.predict(x_train)
train_acc = accuracy_score(x_train_pred,y_train)
print(train_acc)

In [ ]:
x_test_pred = model.predict(x_test)
test_acc = accuracy_score(x_test_pred,y_test)
print(test_acc)

In [ ]:
# Saving the trained model

import pickle
pickle.dump(moedl, open('model.pkl', 'wb'))

In [ ]:
# Loading the saved model

model = pickle.load(open('model.pkl', 'rb'))